# Загрузка

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/it-purple-hack-alpha/cluster_weights.xlsx
/kaggle/input/it-purple-hack-alpha/train_data.pqt
/kaggle/input/it-purple-hack-alpha/sample_submission.csv
/kaggle/input/it-purple-hack-alpha/feature_description.xlsx
/kaggle/input/it-purple-hack-alpha/test_data.pqt


In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [ ]:
train_df = pd.read_parquet('/kaggle/input/it-purple-hack-alpha/train_data.pqt')
test_df = pd.read_parquet('/kaggle/input/it-purple-hack-alpha/test_data.pqt')
train_df.shape, test_df.shape

((600000, 93), (290120, 92))

In [ ]:
test_df_corr = pd.read_parquet('/kaggle/input/it-purple-hack-alpha/test_data.pqt')

# Заполнение фичей

In [ ]:
test_df['start_cluster'].fillna(method='ffill', inplace=True)
test_df['start_cluster'].head(30)

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

In [ ]:
basic_col = ["id", "date", "end_cluster"]
num_feat = [col for col in train_df.columns if (col not in basic_col) and (col not in cat_cols)]

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

In [ ]:
selected_features = cat_cols + num_feat
len(selected_features)

90

In [ ]:
clusters = {'{other}': '{other}',
 '{}': '{}',
 '{α, β}': '{α, β}',
 '{α, γ}': '{α, γ}',
 '{α, δ}': '{α}',
 '{α, ε, η}': '{α, ε, η}',
 '{α, ε, θ}': '{α, ε, θ}',
 '{α, ε, ψ}': '{α, ε, ψ}',
 '{α, ε}': '{α, ε}',
 '{α, η}': '{α, η}',
 '{α, θ}': '{α, θ}',
 '{α, λ}': '{α, λ}',
 '{α, μ}': '{α, μ}',
 '{α, π}': '{other}',
 '{α, ψ}': '{α, ψ}',
 '{α}': '{α}',
 '{λ}': '{α, λ}'}

In [ ]:
train_df['end_cluster_1'] = train_df['start_cluster'].map(clusters)
test_df['end_cluster_1'] = test_df['start_cluster'].map(clusters)

# Обучение моделей

In [ ]:
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

Знаем, что более 300 000 записей имеют стартовый кластер альфа, для них создим отдальную модель

Для остальных - отдельную

In [ ]:
part_1 = train_df[train_df['start_cluster'] == '{α}']

part_2 = train_df[~train_df['start_cluster'].isin(['{α}'])]

train_dfs = [part_1, part_2]
models = []
number = 0
selected_features
for df in train_dfs:
    X = df.drop(["id", "date", "end_cluster"], axis=1)
    y = df["end_cluster"]

    x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                      test_size=0.2,
                                                      random_state=42)
    model = CatBoostClassifier(iterations=1000,
                               verbose=False,
                               learning_rate=0.1,
                               random_state=42,
                               depth=6,
                               cat_features=cat_feat,
                               task_type="GPU",
                               loss_function='MultiClass')
    model.fit(x_train[selected_features], y_train)
    y_pred_proba = model.predict_proba(x_val[selected_features])
    wr = weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)
    print(f'Номер датафрейма  {number} w_roc_auc  {wr}')
    models.append(model)
    number += 1

Номер датафрейма  0 w_roc_auc  0.8852152558538978
Номер датафрейма  1 w_roc_auc  0.9499420664287692


In [ ]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [ ]:
cluster_weights = pd.read_excel("/kaggle/input/it-purple-hack-alpha/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

К сожалению, не успели доработать, но идея кажется перспективной